In [2]:
%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoTokenizer

NER_TAGS_COL = 'tags'

raw_datasets = load_dataset("tner/mit_restaurant")
model_checkpoint = "FacebookAI/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

original_label_to_id = {
    "O": 0,
    "B-Rating": 1,
    "I-Rating": 2,
    "B-Amenity": 3,
    "I-Amenity": 4,
    "B-Location": 5,
    "I-Location": 6,
    "B-Restaurant_Name": 7,
    "I-Restaurant_Name": 8,
    "B-Price": 9,
    "B-Hours": 10,
    "I-Hours": 11,
    "B-Dish": 12,
    "I-Dish": 13,
    "B-Cuisine": 14,
    "I-Price": 15,
    "I-Cuisine": 16
}
original_id_to_label = {i:name for name, i in original_label_to_id.items()}

label2id = {
    "O": 0,
    "B-Rating": 1,
    "I-Rating": 2,
    "B-Amenity": 3,
    "I-Amenity": 4,
    "B-Location": 5,
    "I-Location": 6,
    "B-Restaurant_Name": 7,
    "I-Restaurant_Name": 8,
    "B-Price": 9,
    "I-Price": 10,
    "B-Hours": 11,
    "I-Hours": 12,
    "B-Dish": 13,
    "I-Dish": 14,
    "B-Cuisine": 15,
    "I-Cuisine": 16
}
id2label = {i:name for name, i in label2id.items()}

original_to_ours = {original_label_to_id[k]: label2id[k] for k in label2id}


Using the latest cached version of the dataset since tner/mit_restaurant couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mit_restaurant' at C:\Users\KonstantinGrigorov\.cache\huggingface\datasets\tner___mit_restaurant\mit_restaurant\1.0.0\538663410a86a70f788b0c193d42320de330cc0d (last modified on Fri Jun 28 16:31:29 2024).
C:\Users\KonstantinGrigorov\anaconda3\envs\Own_AI\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
def relable(examples, original_to_ours):
    new_tags = []
    for tags in examples[NER_TAGS_COL]:
        new_tags.append([original_to_ours[item] for item in tags])
    examples[NER_TAGS_COL] = new_tags
    return examples

raw_datasets=raw_datasets.map(relable, batched=True, fn_kwargs={'original_to_ours':original_to_ours})

In [4]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
labels = raw_datasets["train"][0][NER_TAGS_COL]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 9, 15, 0, 5]
[-100, 0, 0, 0, 0, 0, 9, 15, 16, 16, 0, 5, -100]


In [5]:
tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)

{'input_ids': [0, 64, 47, 465, 162, 5, 21084, 162, 1178, 12657, 2391, 3027, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.decode ([0, 64, 47, 465, 162, 5, 21084, 162, 1178, 12657, 2391, 3027, 2])

'<s> can you find me the cheapest mexican restaurant nearby</s>'

In [7]:
# now on the whole dataset
def tokenize_and_align_labels(examples, is_split_into_words=True):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=is_split_into_words
    )
    all_labels = examples[NER_TAGS_COL]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets =raw_datasets.map(tokenize_and_align_labels, batched=True, remove_columns=raw_datasets['train'].column_names)

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    9,   15,   16,   16,    0,    5,
         -100],
        [-100,    0,    0,    0,    0,    0,    5,    7,    8, -100, -100, -100,
         -100]])

In [10]:
import evaluate
import numpy as np
metric = evaluate.load("seqeval")
def ner_metrics_factory(id2label: dict | list, module: str = "seqeval"):
    
    metric = evaluate.load(module)
    def compute_metrics(eval_preds):
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)

        # Remove ignored index (special tokens) and convert to labels
        true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
        true_predictions = [
            [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
        return all_metrics
    
    return compute_metrics

compute_metrics = ner_metrics_factory(id2label)

Using the latest cached version of the module from C:\Users\KonstantinGrigorov\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--seqeval\541ae017dc683f85116597d48f621abc7b21b88dc42ec937c71af5415f0af63c (last modified on Fri Jun 28 16:08:58 2024) since it couldn't be found locally at evaluate-metric--seqeval, or remotely on the Hugging Face Hub.


In [11]:
labels = raw_datasets["train"][0][NER_TAGS_COL]
labels = [id2label[i] for i in labels]
labels

['O', 'O', 'O', 'O', 'O', 'B-Price', 'B-Cuisine', 'O', 'B-Location']

In [12]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (L

In [14]:
from transformers import TrainingArguments
from transformers import Trainer

args = TrainingArguments(
    "roberta-ner",
    evaluation_strategy="epoch",
    save_strategy='no', #epoch
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"], #.select(range(100)),
    eval_dataset=tokenized_datasets["validation"], #.select(range(100)),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

C:\Users\KonstantinGrigorov\anaconda3\envs\Own_AI\Lib\site-packages\accelerate\accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [16]:
trainer.save_model('model/local_ner')

MemoryError: 

In [1]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "model/roberta-ner"
local_checkpoint = 'model/local_ner' # <--------------
token_classifier = pipeline(
    "token-classification", model=local_checkpoint, aggregation_strategy="average"
)
token_classifier("Can you locate a place to eat that has steak on the menu and serves breakfast all day")

C:\Users\KonstantinGrigorov\anaconda3\envs\Own_AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\KonstantinGrigorov\anaconda3\envs\Own_AI\Lib\site-packages\transformers\pipelines\token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


[{'entity_group': 'Dish',
  'score': 0.5414536,
  'word': ' steak',
  'start': 39,
  'end': 44},
 {'entity_group': 'Amenity',
  'score': 0.27536303,
  'word': ' breakfast',
  'start': 68,
  'end': 77},
 {'entity_group': 'Hours',
  'score': 0.5469777,
  'word': ' all day',
  'start': 78,
  'end': 85}]